**余盈蓓；語碩二；110555009**\
**111-2 Computational Linguistics**
<p align="center", style = "font-size:18pt">
<b>7th Assignment<br>
Deep Learning</b>
</p>

<p style = "font-size:14pt">
<b>Step 1. 資料讀入</b>
</p>

In [10]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace

# preprocess the dataset: remove all the <br /> tag

def prepareData(dir):
  data = text_dataset_from_directory(dir) # load in dataset
  return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label),
  )


# load in dataset
train_data = prepareData("movie-reviews-dataset/train")
test_data = prepareData("movie-reviews-dataset/test")

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [12]:
for text_batch, label_batch in train_data.take(2):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0]) # 0 = negative, 1 = positive

b"This is the movie that I use to judge all other bad movies, and so far there hasn't been anything close.  The only good thing I can say is that after watching this I know that I have seen the worst movie I will ever see."
0
b'The real star of this ridiculous story is glorious technicolor. A visual treat to the eye, the film fails to stimulate the mind and heart. I was intrigued, at first, by the idea of Dietrich and Boyer leaving religion in order to "find" their capacity for love. What follows is a huge disappointment. Boyer is the only real actor in the production and one feels his torment. Dietrich\'s amazing wardrobe outshines her performance -- at times her face is frightening to look at -- a unfeeling mask. As a monk, Boyer held the formula for the monastery\'s liquer (which reminds me of the true story of Chartreuse) -- when he leaves his "marriage to god" the reaction by his fellow monks holds the shock and fear that perpetuate organized religion. The viewer feels Boyer was w

<p style = "font-size:14pt">
<b>Step 2. 向量化文字</b>
</p>

In [15]:

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

max_tokens = 1000
max_len = 50
vectorize_layer = TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_len,
)

train_texts = train_data.map(lambda text, label: text) 
vectorize_layer.adapt(train_texts)

<p style = "font-size:14pt">
<b>Step 3. 模型訓練</b>
</p>
<p style = "font-size:12pt">
✨ <b>GRU</b><br>
&emsp;&ensp;→ 初始化模型
</p>

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input

model = Sequential(name="GRU-model")
model.add(Input(shape=(1,), dtype="string"))
model.add(vectorize_layer)

<p style = "font-size:12pt">
&emsp;&ensp;→ 加入embedding
</p>

In [30]:
from tensorflow.keras.layers import Embedding

model.add(Embedding(max_tokens + 1, 128))

<p style = "font-size:12pt">
&emsp;&ensp;→ 加入GRU layer
</p>

In [31]:
from tensorflow.keras.layers import GRU

model.add(GRU(64))

<p style = "font-size:12pt">
&emsp;&ensp;→ 加入一個hidden layer跟最後的output layer
</p>

In [32]:
from tensorflow.keras.layers import Dense

model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

<p style = "font-size:12pt">
&emsp;&ensp;→ 完成模型
</p>

In [33]:
model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy'],
)

In [34]:
print(model.summary())

Model: "GRU-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 50)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 50, 128)           128128    
                                                                 
 gru_2 (GRU)                 (None, 64)                37248     
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 169,601
Trainable params: 169,601
Non-trainable params: 0
___________________________________________________

<p style = "font-size:12pt">
&emsp;&ensp;→ 放入訓練集訓練模型
</p>

In [35]:
model.fit(train_data, epochs=50)

Epoch 1/50
782/782 [==============================] - 14s 16ms/step - loss: 0.5833 - accuracy: 0.6781
Epoch 2/50
782/782 [==============================] - 13s 16ms/step - loss: 0.5028 - accuracy: 0.7532
Epoch 3/50
782/782 [==============================] - 27s 35ms/step - loss: 0.4714 - accuracy: 0.7702
Epoch 4/50
782/782 [==============================] - 33s 43ms/step - loss: 0.4499 - accuracy: 0.7810
Epoch 5/50
782/782 [==============================] - 28s 35ms/step - loss: 0.4336 - accuracy: 0.7891
Epoch 6/50
782/782 [==============================] - 26s 33ms/step - loss: 0.4170 - accuracy: 0.7977
Epoch 7/50
782/782 [==============================] - 25s 32ms/step - loss: 0.3996 - accuracy: 0.8035
Epoch 8/50
782/782 [==============================] - 25s 32ms/step - loss: 0.3801 - accuracy: 0.8108
Epoch 9/50
782/782 [==============================] - 26s 33ms/step - loss: 0.3633 - accuracy: 0.8193
Epoch 10/50
782/782 [==============================] - 25s 32ms/step - loss: 0.341

<p style = "font-size:12pt">
✨ <b>bi-LSTM</b><br>
&emsp;&ensp;→ 設定模型參數
</p>

In [90]:
from tensorflow.keras.layers import LSTM, Bidirectional

bilstm_model = Sequential([
    Input(shape=(1,), dtype="string"),
    vectorize_layer,
    Embedding(max_tokens + 1, 128),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
bilstm_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(bilstm_model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 50)               0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, 50, 128)           128128    
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 235,265
Trainable params: 235,265
Non-tr

<p style = "font-size:12pt">
&emsp;&ensp;→ 放入訓練集訓練模型
</p>

In [91]:
bilstm_model.fit(train_data, epochs=50)

Epoch 1/50
782/782 [==============================] - 19s 22ms/step - loss: 0.5535 - accuracy: 0.7054
Epoch 2/50
782/782 [==============================] - 23s 29ms/step - loss: 0.4865 - accuracy: 0.7562
Epoch 3/50
782/782 [==============================] - 25s 32ms/step - loss: 0.4550 - accuracy: 0.7766
Epoch 4/50
782/782 [==============================] - 28s 36ms/step - loss: 0.4245 - accuracy: 0.7968
Epoch 5/50
782/782 [==============================] - 35s 45ms/step - loss: 0.3916 - accuracy: 0.8151
Epoch 6/50
782/782 [==============================] - 36s 46ms/step - loss: 0.3598 - accuracy: 0.8320
Epoch 7/50
782/782 [==============================] - 36s 45ms/step - loss: 0.3292 - accuracy: 0.8471
Epoch 8/50
782/782 [==============================] - 35s 45ms/step - loss: 0.2988 - accuracy: 0.8619
Epoch 9/50
782/782 [==============================] - 35s 45ms/step - loss: 0.2735 - accuracy: 0.8744
Epoch 10/50
782/782 [==============================] - 34s 43ms/step - loss: 0.247

<p style = "font-size:12pt">
✨ <b>bi-RNN</b><br>
&emsp;&ensp;→ 設定模型參數
</p>

In [92]:
from tensorflow.keras.layers import SimpleRNN

birnn_model = Sequential([
    Input(shape=(1,), dtype="string"),
    vectorize_layer,
    Embedding(max_tokens + 1, 128),
    Bidirectional(SimpleRNN(64)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
birnn_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(birnn_model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 50)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 50, 128)           128128    
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              24704     
 nal)                                                            
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 161,153
Trainable params: 161,153
Non-tr

<p style = "font-size:12pt">
&emsp;&ensp;→ 放入訓練集訓練模型
</p>

In [93]:
birnn_model.fit(train_data, epochs=50)

Epoch 1/50
782/782 [==============================] - 12s 14ms/step - loss: 0.6478 - accuracy: 0.6045
Epoch 2/50
782/782 [==============================] - 14s 18ms/step - loss: 0.5452 - accuracy: 0.7261
Epoch 3/50
782/782 [==============================] - 22s 28ms/step - loss: 0.5029 - accuracy: 0.7562
Epoch 4/50
782/782 [==============================] - 26s 33ms/step - loss: 0.4735 - accuracy: 0.7752
Epoch 5/50
782/782 [==============================] - 25s 32ms/step - loss: 0.4223 - accuracy: 0.8080
Epoch 6/50
782/782 [==============================] - 28s 35ms/step - loss: 0.3677 - accuracy: 0.8386
Epoch 7/50
782/782 [==============================] - 26s 33ms/step - loss: 0.3173 - accuracy: 0.8651
Epoch 8/50
782/782 [==============================] - 26s 34ms/step - loss: 0.2774 - accuracy: 0.8820
Epoch 9/50
782/782 [==============================] - 30s 38ms/step - loss: 0.2384 - accuracy: 0.9024
Epoch 10/50
782/782 [==============================] - 24s 31ms/step - loss: 0.214

<p style = "font-size:14pt">
<b>Step 4. 保存模型參數</b>
</p>

In [95]:
model.save_weights('GRU')
bilstm_model.save_weights('bi-LSTM')
birnn_model.save_weights('bi-RNN')

<p style = "font-size:14pt">
<b>Step 5. 模型評估</b>
</p>
<p style = "font-size:12pt">
✨ <b>GRU</b><br>
</p>

In [107]:
GRU_results = model.evaluate(test_data)
print("test loss, test acc:", GRU_results)

782/782 [==============================] - 7s 9ms/step - loss: 2.8272 - accuracy: 0.6918
test loss, test acc: [2.827223539352417, 0.6917999982833862]


<p style = "font-size:12pt">
✨ <b>bi-LSTM</b><br>
</p>

In [108]:
LSTM_results = bilstm_model.evaluate(test_data)
print("test loss, test acc:", LSTM_results)

782/782 [==============================] - 10s 13ms/step - loss: 2.9130 - accuracy: 0.6950
test loss, test acc: [2.9129574298858643, 0.6949599981307983]


<p style = "font-size:12pt">
✨ <b>bi-RNN</b><br>
</p>

In [109]:
RNN_results = birnn_model.evaluate(test_data)
print("test loss, test acc:", RNN_results)

782/782 [==============================] - 6s 8ms/step - loss: 2.0232 - accuracy: 0.6617
test loss, test acc: [2.0231945514678955, 0.6616799831390381]


<p align = "center", style = "font-size:16pt">
    🧐<b>討論</b>🧐<br>
</p>
<p style = "font-size:12pt">
✨分數統整<br>
</p>
<table>
  <tr>
    <th rowspan = "2">&emsp;&emsp;</th>
    <th colspan = "2", style = "background-color: #F1E1FF" >training</th>
    <th colspan = "2", style = "background-color: #F1E1FF" >testing</th>
  </tr>
  <tr style = "background-color: #FFECF5">
    <th>loss</th>
    <th>accuracy</th>
    <th>loss</th>
    <th>accuracy</th>
  </tr>
  <tr>
    <th style = "background-color: 	#D2E9FF">GRU</th>
    <td>0.0297</td>
    <td>0.9907</td>
    <td>2.8272</td>
    <td>0.6918</td>
  </tr>
  <tr>
    <th style = "background-color: 	#D2E9FF">bi-RNN</th>
    <td>0.0539</td>
    <td>0.9812</td>
    <td>2.0232</td>
    <td>0.6617</td>
  </tr>
  <tr>
    <th style = "background-color: 	#D2E9FF">bi-LSTM</th>
    <td >0.0158</td>
    <td>0.9945</td>
    <td>2.9130</td>
    <td>0.6950</td>
  </tr>
</table>

<p style = "font-size:12pt">
   &emsp;&emsp;首先從模型訓練的過程就可以看到，在參數設定都一樣的情況下，bi-LSTM的表現是最好的。而測試的結果也是bi-LSTM的sccuracy最高，唯一不同是bi-LSTM的loss是最高的。bi-LSTM的表現最好其實是可以預見的結果，因為它改善了RNN可能把長句子中比較前面的context稀釋掉的問題，又比GRU多了一個方向去考慮更多context，所以結果才是最好的。比較讓人意外的是他們之間的差距其實沒有很大，這個部分不確定是這幾個模型其實都只是在細節上針對缺點做了一點修改的關係，還是資料的問題。
</p>